In [1]:
import pandas as pd
from rdkit import Chem
import seaborn as sns
from tqdm import tqdm
import numpy as np

In [2]:
tqdm.pandas()


In [25]:
chembl = pd.read_csv("ChEMBL/ChEMBL3401.tsv", sep="\t")

In [26]:
chembl

,ChEMBL ID,Name,Synonyms,Type,Max Phase,Molecular Weight,Targets,Bioactivities,AlogP,Polar Surface Area,...,Np Likeness Score,Molecular Species,Molecular Formula,Smiles,Inchi Key,Inchi,Withdrawn Flag,Orphan,Records Key,Records Name
0,CHEMBL48361,DABIGATRAN,BIBR 953 ZW|BIBR-953|BIBR-953-ZW|DABIGATRAN|PR...,Small molecule,3.0,471.52,88,164,2.99,150.22,...,-1.54,ZWITTERION,C25H25N7O3,Cn1c(CNc2ccc(C(=N)N)cc2)nc2cc(C(=O)N(CCC(=O)O)...,YBSJFWOBGCMAKL-UHFFFAOYSA-N,InChI=1S/C25H25N7O3/c1-31-20-10-7-17(25(35)32(...,False,0,"['Dabigatran', '24, Dabigatran', 'DABIGATRAN',...","['Dabigatran', '3-({2-[(4-Carbamimidoyl-phenyl..."
1,CHEMBL492,ETIDOCAINE,ETIDOCAINA|ETIDOCAINE|Etidocaine|W-19053|W-190...,Small molecule,4.0,276.42,69,88,3.75,32.34,...,-1.39,BASE,C17H28N2O,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C,VTUSIVBDOCDNHS-UHFFFAOYSA-N,InChI=1S/C17H28N2O/c1-6-12-19(8-3)15(7-2)17(20...,False,0,"['35', 'Etidocaine', 'SID144206521', '26', 'Et...","['N-(2,6-Dimethyl-phenyl)-2-(ethyl-propyl-amin..."
2,CHEMBL878,METOLAZONE,DIULO|METAZOLINE|METENIX|METOLAZONA|METOLAZONE...,Small molecule,4.0,365.84,165,562,2.71,92.50,...,-1.18,NEUTRAL,C16H16ClN3O3S,Cc1ccccc1N1C(=O)c2cc(S(N)(=O)=O)c(Cl)cc2NC1C,AQCHWTWZEMGIFD-UHFFFAOYSA-N,InChI=1S/C16H16ClN3O3S/c1-9-5-3-4-6-14(9)20-10...,False,0,"['Metolazone', 'Metolazone', 'Metolazone', 'NC...","['7-Chloro-2-methyl-4-oxo-3-o-tolyl-1,2,3,4-te..."
3,CHEMBL1371200,CANRENOATE POTASSIUM,CANRENOATE DE POTASSIUM|CANRENOATE POTASSIUM|C...,Small molecule,4.0,396.57,65,80,3.89,74.60,...,2.38,ACID,C22H29KO4,C[C@]12CCC(=O)C=C1C=C[C@@H]1[C@@H]2CC[C@@]2(C)...,JTZQCHFUGHIPDF-RYVBEKKQSA-M,InChI=1S/C22H30O4.K/c1-20-9-5-15(23)13-14(20)3...,False,0,"['CANRENOATE POTASSIUM', 'CANRENOIC ACID, POTA...","['CANRENOATE POTASSIUM', 'CANRENOIC ACID, POTA..."
4,CHEMBL847,OXAMNIQUINE,NSC-352888|OXAMNIQUINA|OXAMNIQUINE|OXAMNIQUINU...,Small molecule,4.0,279.34,85,137,1.81,87.43,...,-0.22,BASE,C14H21N3O3,CC(C)NCC1CCc2cc(CO)c([N+](=O)[O-])cc2N1,XCGYUJZMCCFSRP-UHFFFAOYSA-N,InChI=1S/C14H21N3O3/c1-9(2)15-7-12-4-3-10-5-11...,False,0,"['OXAMNIQUINE', '1 (oxamniquine)', 'SID2666519...","['OXAMNIQUINE', '[2-(Isopropylamino-methyl)-7-..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3543,CHEMBL66092,LAQUINIMOD,ABR-215062|ABR-215062 FREE ACID|LAQUINIMOD|TV-...,Small molecule,3.0,356.81,26,40,3.56,62.54,...,-1.08,ACID,C19H17ClN2O3,CCN(C(=O)c1c(O)c2c(Cl)cccc2n(C)c1=O)c1ccccc1,GKWPCEFFIHSJOE-UHFFFAOYSA-N,InChI=1S/C19H17ClN2O3/c1-3-22(12-8-5-4-6-9-12)...,False,0,"['LAQUINIMOD', '8c', 'SID174007071', 'Laquinim...","['LAQUINIMOD', '5-Chloro-4-hydroxy-1-methyl-2-..."
3544,CHEMBL178,DAUNORUBICIN,Cerubidine|DAUNOMYCIN|DAUNORUBICIN|DAUNORUBICI...,Small molecule,4.0,527.53,293,2055,1.03,185.84,...,1.75,BASE,C27H29NO10,COc1cccc2c1C(=O)c1c(O)c3c(c(O)c1C2=O)C[C@@](O)...,STQGQHZAVUOBTE-VGBVRHCVSA-N,InChI=1S/C27H29NO10/c1-10-22(30)14(28)7-17(37-...,False,0,"['5', '1', 'Daunorubicin', '1', 'DR', 'Daunoru...","['(8S,10S)-8-Acetyl-10-(4-amino-5-hydroxy-6-me..."
3545,CHEMBL1201313,CHLOPHEDIANOL,ANTITUSSIN|CLOFEDANOL|Chlophedianol|NSC-113595...,Small molecule,4.0,289.81,63,72,3.53,23.47,...,-0.79,BASE,C17H20ClNO,CN(C)CCC(O)(c1ccccc1)c1ccccc1Cl,WRCHFMBCVFFYEQ-UHFFFAOYSA-N,"InChI=1S/C17H20ClNO/c1-19(2)13-12-17(20,14-8-4...",False,0,"['SID144206265', 'SID93577132', 'CHLOPHEDIANOL...","['SID144206265', 'SID93577132', 'CHLOPHEDIANOL..."
3546,CHEMBL744,RILUZOLE,BHV-0223|EXSERVAN|NSC-753433|NSC-759823|RILUTE...,Small molecule,4.0,234.20,225,778,2.78,48.14,...,-2.02,NEUTRAL,C8H5F3N2OS,Nc1nc2ccc(OC(F)(F)F)cc2s1,FTALBRSUTCGOEG-UHFFFAOYSA-N,"InChI=1S/C8H5F3N2OS/c9-8(10,11)14-4-1-2-5-6(3-...",False,0,"['11', 'Riluzole', '1 - Riluzole', '24', '1', ...",['6-Trifluoromethoxy-benzothiazol-2-ylamine (R...


In [27]:
chembl.columns

Index(['ChEMBL ID', 'Name', 'Synonyms', 'Type', 'Max Phase',
       'Molecular Weight', 'Targets', 'Bioactivities', 'AlogP',
       'Polar Surface Area', 'HBA', 'HBD', '#RO5 Violations',
       '#Rotatable Bonds', 'Passes Ro3', 'QED Weighted', 'CX Acidic pKa',
       'CX Basic pKa', 'CX LogP', 'CX LogD', 'Aromatic Rings',
       'Structure Type', 'Inorganic Flag', 'Heavy Atoms', 'HBA (Lipinski)',
       'HBD (Lipinski)', '#RO5 Violations (Lipinski)',
       'Molecular Weight (Monoisotopic)', 'Np Likeness Score',
       'Molecular Species', 'Molecular Formula', 'Smiles', 'Inchi Key',
       'Inchi', 'Withdrawn Flag', 'Orphan', 'Records Key', 'Records Name'],
      dtype='object')

In [31]:
n_mol = chembl["ChEMBL ID"].nunique()
n_mol

3548

In [33]:

# from Pat Walters method
def standardize_smiles(smiles):
    try:
        # follows the steps in
        # https://github.com/greglandrum/RSC_OpenScience_Standardization_202104/blob/main/MolStandardize%20pieces.ipynb
        # as described **excellently** (by Greg) in
        # https://www.youtube.com/watch?v=eWTApNX8dJQ
        mol = Chem.MolFromSmiles(smiles)
    
        # removeHs, disconnect metal atoms, normalize the molecule, reionize the molecule
        clean_mol = rdMolStandardize.Cleanup(mol)
    
        # if many fragments, get the "parent" (the actual mol we are interested in) 
        parent_clean_mol = rdMolStandardize.FragmentParent(clean_mol)
    
        # try to neutralize molecule
        uncharger = rdMolStandardize.Uncharger()  # annoying, but necessary as no convenience method exists
        uncharged_parent_clean_mol = uncharger.uncharge(parent_clean_mol)
    
        # note that no attempt is made at reionization at this step
        # nor at ionization at some pH (rdkit has no pKa caculator)
        # the main aim to represent all molecules from different sources
        # in a (single) standard way, for use in ML, catalogue, etc.
    
        te = rdMolStandardize.TautomerEnumerator()  # idem
        taut_uncharged_parent_clean_mol = te.Canonicalize(uncharged_parent_clean_mol)
    
        return Chem.MolToSmiles(taut_uncharged_parent_clean_mol)
    except:
        return pd.NA

In [34]:
# standardisation of SMILES
from rdkit import Chem
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.rdBase import BlockLogs
import mols2grid

In [35]:

with BlockLogs():
    chembl["CANONICAL_SMILES"] = chembl["Smiles"].progress_apply(lambda x: standardize_smiles(x))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3548/3548 [00:32<00:00, 108.15it/s]


In [36]:
def smi_to_inchikey(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        return Chem.MolToInchiKey(mol)
    except:
        return pd.NA

In [37]:
# compute inchikeys
with BlockLogs():
    chembl["INCHIKEY"] = chembl["CANONICAL_SMILES"].progress_apply(lambda x: smi_to_inchikey(x))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3548/3548 [00:01<00:00, 3140.71it/s]


In [38]:
chembl.INCHIKEY.nunique()

3524

In [39]:
chembl.dropna(subset="INCHIKEY", inplace=True)

In [49]:
# now clean up 
keep_cols = ["CANONICAL_SMILES", "INCHIKEY", "pChEMBL mean", "pChEMBL std", "Molecule Name", "assay_count", "Action Type"]

In [41]:
clean_active = chembl[keep_cols]
clean_active.rename(columns={ "Name": "common_name"}, inplace=True)



/tmp/ipykernel_313175/3609579136.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_active.rename(columns={ "Name": "common_name"}, inplace=True)


In [42]:
clean_active_sorted = clean_active.sort_values(["common_name"], ascending=[False]) # keep the ones with names if possible
clean_deduped = clean_active_sorted.drop_duplicates(subset="INCHIKEY", keep="first")

In [43]:
clean_active.head()

,CANONICAL_SMILES,INCHIKEY,common_name
0,Cn1c(CNc2ccc(C(=N)N)cc2)nc2cc(C(=O)N(CCC(=O)O)...,YBSJFWOBGCMAKL-UHFFFAOYSA-N,DABIGATRAN
1,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C,VTUSIVBDOCDNHS-UHFFFAOYSA-N,ETIDOCAINE
2,Cc1ccccc1N1C(=O)c2cc(S(N)(=O)=O)c(Cl)cc2NC1C,AQCHWTWZEMGIFD-UHFFFAOYSA-N,METOLAZONE
3,C[C@]12CCC(=O)C=C1C=CC1[C@@H]2CC[C@@]2(C)[C@H]...,PBKZPPIHUVSDNM-PVAIODFMSA-N,CANRENOATE POTASSIUM
4,CC(C)NCC1CCc2cc(CO)c([N+](=O)[O-])cc2N1,XCGYUJZMCCFSRP-UHFFFAOYSA-N,OXAMNIQUINE


In [46]:
clean_deduped["dataset"] = "ChEMBL_curated"
clean_deduped["active"] = True

/tmp/ipykernel_313175/2487547738.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_deduped["dataset"] = "ChEMBL_curated"
/tmp/ipykernel_313175/2487547738.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_deduped["active"] = True


In [48]:
clean_deduped.to_csv("processed/chembl_active.csv", index=False)